# TensorFlow Basics

__Notes__:Eager execution is a means of using tensorflow more like regular python code and is a great way to test out code in a notebook. We will avoid using eager execution to learn the paradigm tensorflow uses and to prepare ourselves for acutal implementation. For more information on this eager executions see: https://medium.com/coding-blocks/eager-execution-in-tensorflow-a-more-pythonic-way-of-building-models-e461810618c8 
___________________________________________

The following tutorial goes through some of the basic concepts used in tensorflow, it is more a tutorial than an interactive worksop. It will cover the topics of:

- Computational Graphs
- Tensors and tensor types in tensorflow
- Sessions

If you are comfortable with these concepts please move on.

# Graph based programming

Additional Information:
https://www.tensorflow.org/guide/graphs


The biggest idea about Tensorflow is that all the numerical computations are expressed as a computational graph. In other words, the backbone of any Tensorflow program is a Graph. Anything that happens in your model is represented by the computational graph. This makes it, the to go place for anything related to your model. Quoted from the TensorFlow website, "A computational graph (or graph in short) is a series of TensorFlow operations arranged into a graph of nodes". Basically, it means a graph is just an arrangement of nodes that represent the operations in your model.

So First let's see what does a node and operation mean? The best way to explain it is by looking at a simple example. Suppose we want to write the code for function $f(x,y)=x^2y+y+2$. The Graph in TensorFlow will be something like:![SimpleGraph](simple_graph.png "SimpleGraph")


The graph is composed of a series of nodes connected to each other by edges (from the image above). Each node in the graph is called op (short for operation). So we'll have one node for each operation; either for operations on tensors (like math operations) or generating tensors (like variables and constants). Each node takes zero or more tensors as inputs and produces a tensor as an output.

Now Let's build a simple computational graph.

Example 1:
Let's start with a basic arithmatic operation like addition to demonstrate a graph. The code adds two values, say a=2 and b=3, using TensorFlow. To do so, we need to call tf.add(). From here on, we recommend you to check out the documentation of each method/class to get a clear idea of what it can do(documentation can be found at tensorflow.org or you can just use google to get to the required page in the documentation). The tf.add() has three arugments 'x', 'y', and 'name' where x and y are the values to be added together and name is the operation name, i.e. the name associated to the addition node on the graph.

If we call the operation "Add", the code will be as follows:

In [ ]:
import tensorflow as tf
a = 2
b = 3
c = tf.add(a, b, name='Add')
print(c)

 The generated graph and variables are:
 
 ![GraphNode](graph_node.png "GraphNode")

This code creates two input nodes (for inputs a=2 and b=3) and one output node for the addition operation (named Add). When we print out the variable c (i.e. the output Tensor of the addition operation), it prints out the Tensor information; its name (Add), shape (() means scalar), and type (32-bit integer). However, It does not spit out the result (2+3=5). Why?!

Remember earlier in this post, we talked about the two parts of a TensorFlow code. First step is to create a graph and to actually evaluate the nodes, we must run the computational graph within a Session. In simple words, the written code only generates the graph which only determines the expected sizes of Tensors and operations to be executed on them. However, it doesn't assign a numeric value to any of the Tensors i.e. TensorFlow does not execute the graph unless it is specified to do so with a session. Hence, to assign these values and make them flow through the graph, we need to create and run a session.

Therefore a TensorFlow Graph is something like a function definition in Python. It WILL NOT do any computation for you (just like a function definition will not have any execution result). It ONLY defines computation operations.

## Session
To compute anything, a graph must be launched in a session. Technically, session places the graph ops on hardware such as CPUs or GPUs and provides methods to execute them. In our example, to run the graph and get the value for c the following code will create a session and execute the graph by running 'c':

In [ ]:
sess = tf.Session()
print(sess.run(c))
sess.close()

This code creates a Session object (assigned to __sess__), and then (the second line) invokes its run method to run enough of the computational graph to evaluate __c__. This means that it only runs that part of the graph which is necessary to get the value of __c__ (remember the flexibility of using TensorFlow? In this simple example, it runs the whole graph). Remember to close the session at the end of the session. That is done using the last line in the above code. 

The following code does the same thing and is more commonly used. The only difference is that there is no need to close the session at the end as it gets closed automatically.

In [ ]:
with tf.Session() as sess:
    print(sess.run(c))

Now let's look at the created graph one more time. Don't you see anything weird?
<img src="files/files/1_4.png" width="500" height="1000">

___Fig4. ___ The generated graph visualized by Tensorboard


Exactly! What is x and y?! Where did these two thing come from? We didn't define any x or y variables!

Well... To explain clearly, let's make up two names; say __"Python-name"__ and __"TensorFlow-name"__. In this piece of code, we generated 3 variables (look at the right panel of Fig. 3) with __"Python-name"__s of _a_, _b_, and _c_. Here, _a_ and _b_ are Python variables, thus have no __"TensorFlow-name"__; while _c_ is a Tensor with ___Add___ as its __"TensorFlow-name"__. 

Clear? Okay, let's get back to our question, what is x and y then?

In an ideal Tensorflow case, __tf.add()__ receives two __Tensors__ with defined __"TensorFlow-name"__ as input (these names are separate from __Python-name__). For example, by writing $c = tf.add(a, b, name='Add')$, we're actually creating a variable (or Tensor) with __c__ as its Python-name and __Add__ as the TensorFlow-name. 

In the above code, we passed two Python variables (a=2 and b=3) which only have Python-names (a and b), but they have no TensorFlow-names. TensorFlow uses the TensorFlow-names for visualizing the graphs. Since a and b have no TensorFlow-names, it uses some default names, x and y. 

__*Note:__ This name mismatch can easily be solved by using tf.constant() for creating the input variables as Tensors instead of simply using Python variables (a=2, b=3). This is explained thoroughly in the next tutorial where we talk about TensorFlow DataTypes. 

For now, we'll continue using Python variables and change the Python variable names __a__ and __b__ into __x__ and __y__ to solve the name mismatch temporarily. 

Now let's look at a more complicated example.

### Example 2:
Creating a graph with multiple math operations

In [ ]:
import tensorflow as tf
x = 2
y = 3
add_op = tf.add(x, y, name='Add')
mul_op = tf.multiply(x, y, name='Multiply')
pow_op = tf.pow(add_op, mul_op, name='Power')
useless_op = tf.multiply(x, add_op, name='Useless')

with tf.Session() as sess:
    pow_out, useless_out = sess.run([pow_op, useless_op])
    
print(pow_out,useless_out)

The created graph and the defined variables (Tensors and Python variables) are:
<img src="files/files/1_5.png" width="1000" height="2000">

___Fig5. ___ __Left:__ generated graph visualized in Tensorboard, __Right:__ generated variables (screenshot captured from PyCharm debugger when running in debug mode)


I called one of the operations useless_op because it's output is not used by other operations. Lets talk about an __IMPORTANT__ point. Given this graph, if we fetch the __pow_op__ operation, it will first run the __add_op__ and __mul_op__ to get their output tensor and then run __pow_op__ on them to compute the required output value. In other words __useless_op__ will not be executed as it's output tensor is not used in executing the __pow_op__ operation. 

__This is one of the advantages of defining a graph and running a session on it! It helps running only the required operations of the graph and skip the rest (remember flexibility). This specially saves a significant amount of time for us when dealing with huge networks with hundreds and thousands of operations.__

In the above code, in the defined session, we're fetching the value of two tensors (i.e. output tensors of __pow_op__ and __useless_op__) at the same time. This will run the whole graph to get the required output tensors.

# Exercies

Create computational graphs and execute the following operations:

1\. $$ \log(0.001) $$

2\. $$ 5^2 +  7$$

3\. $$\frac{\pi}{e^{-2}} $$

4\. $$5 > 10 $$


__Note__: The computational graph builds upon every tensorflow command you execute whether those operations are connected or not. To erase the existing graph and start from scratch use the command:

In [ ]:
tf.reset_default_graph()

# Tensor Types

In the previous post, we read about the concepts of __Graph__ and __Session__ which describes the way the data flows in TensorFlow. One of the first questions you might have while learning a new framework is of any new data structure that should used. TensorFlow does have its own data structure for the purpose of performance and ease of use. Tensor is the data structure used in Tensorflow (remember TensorFlow is the flow of tensors in a computational graph) and it is at the core of TensorFlow. TensorFlow programs use a tensor data structure to represent all data — only tensors are passed between operations in the computation graph. You can think of a TensorFlow tensor as an n-dimensional array or list.

In this tutorial, we'll take a look at some of the __Tensor Types__ used in TensorFlow. The speciall ones commonly used in creating neural network models are namely ___Constant___, ___Variable___, and ___Placeholder___. 



# Constants

the name speaks for itself, __Constants__ are used as constants. They create a node that takes value and it does not change. You can simply create a constant tensor using __tf.constant__. It accepts the five arguments:


In [ ]:
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

Now let's take a look at a very simple example.

### Example 1:
Let's create two constants and add them together. Constant tensors can simply be defined with a value:

In [ ]:
# create graph
a = tf.constant(2)
b = tf.constant(3)
c = a + b
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

Perfect! Now Let's look at the created graph and generated data types:
<img src="files/files/2_1.png" width="1000" height="2000" >
___Fig1. ___ __Left:__ generated graph visualized in Tensorboard, __Right:__ generated variables (screenshot captured from PyCharm debugger when running in debug mode)

In the figure, we created 3 tensors with __"Python-names"__ _a_, _b_, and _c_. As we didn't define any __"TensorFlow-name"__ for them, TensorFlow assigns some default names to them which are observed in the graph: __const__ and __const_1__ for the input constants and __add__ for the output of the addition operation. We can easily modify it and define custom names as shown below:

In [ ]:
# create graph
a = tf.constant(2, name='A')
b = tf.constant(3, name='B')
c = tf.add(a, b, name='Sum')
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

This time the graph is created with the required tensor names:
<img src="files/files/2_2.png" width="1000" height="2000" >
___Fig2. ___ generated graph (Left) and variables (Right) with the modified names


Constants can also be defined with different types (integer, float, etc.) and shapes (vectors, matrices, etc.). The next example has one constant with type 32bit float and another constant with shape 2X2.


### Example 2:

In [ ]:
s = tf.constant(2.3, name='scalar', dtype=tf.float32)
m = tf.constant([[1, 2], [3, 4]], name='matrix')
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(s))
    print(sess.run(m))

## 2. Variable

Variables are stateful nodes which output their current value; meaning that they can retain their value over multiple executions of a graph. They have a number of useful features such as:

- They can be __saved__ to your disk during and after training. This allows people from different companies and groups to collaborate as they can save, restore and send over their model parameters to other people. (read more on [Save and Restore Tutorial](https://github.com/easy-tensorflow/easy-tensorflow/blob/master/1_TensorFlow_Basics/Tutorials/4_Save_and_Restore.ipynb))
- By default, gradient updates (used in all neural networks) will apply to all variables in your graph. In fact, variables are the things that you want to tune in order to minimize the loss. 

These two features make variables suitable to be used as the network parameters (i.e. weights and biases). You might ask, what are the differences between variables and constants? Well there are two major differences:

1. Constants are (guess what!), constants. As their name states, their value doesn't change. We'd usually need our network parameters to be updated and that's where the __variable__ comes into play.

2. Constants are stored in the graph definition which makes them memory-expensive. In other words, constants with millions of entries makes the graph slower and resource intensive.


Again, it's important to remember that creating a variables is an operation (look at the Fig. 2 of the first tutorial for a quick recap). We execute these operations in the session and get the output value of the operations.

### 2.1. Create Variables

To create a variable, we should use __tf.Variable__ as:


In [ ]:
# Create a variable.
w = tf.Variable(<initial-value>, name=<optional-name>)

Some examples of creating scalar and matrix variables are as follows:

In [ ]:
s = tf.Variable(2, name="scalar") 
m = tf.Variable([[1, 2], [3, 4]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))

Variable __W__ defined above creates a matrix with 784 rows and 10 columns which will be initialized with zeros. This can be used as a weight matrix of a feed-forward neural network (or even in a linear regression model) from a layer with 784 neuron to a layer with 10 neuron. We'll see more of this later in this turorial.

__*Note:__ We use tf.Variable() with uppercase "V", and tf.constant with lowercase "c". You don't necessarily need to know the reason, but it's simply because tf.constant is an op, while tf.Variable is a class with multiple ops.

__* IMPORTANT Note:__ Calling tf.Variable to create a variable is the old way of creating a variable. TensorFlow recommends to use the wraper __tf.get_variable__ instead as it accepts the name, shape, etc as its arguments with many more as follows:

In [ ]:
tf.get_variable(name,
                shape=None,
                dtype=None,
                initializer=None,
                regularizer=None,
                trainable=True,
                collections=None,
                caching_device=None,
                partitioner=None,
                validate_shape=True,
                use_resource=None,
                custom_getter=None,
                constraint=None)

Some examples are as follows:

In [ ]:
s = tf.get_variable("scalar", initializer=tf.constant(2)) 
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("weight_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

### 2.2. Initialize Variables

Just like most programming languages, Variables need to be initialized before being used. TensorFlow, while not being a language, is no exception to this rule. To initialize variables, we have to invoke a __variable initializer operation__ and run the operation on the session. This is the easiest way to initialize variables all variables at once.

The following toy example shows how we can add an op to initialize the variables.

### Example 3:

Create two variables and add them together. Then print out their values and the summation result. Try running:

In [ ]:
a = tf.get_variable(name="var_1", initializer=tf.constant(2))
b = tf.get_variable(name="var_2", initializer=tf.constant(3))
c = tf.add(a, b, name="Add1")

# launch the graph in a session
with tf.Session() as sess:
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

you get the error: __FailedPreconditionError: Attempting to use uninitialized value__


Upon executing the program, we run into __FailedPreconditionError: Attempting to use uninitialized value__. This is because we tried to evaluate the variables before initializing them. Let's correct the code by first initializing all the variables and then proceed to evaluate them.

In [ ]:
# create graph
a = tf.get_variable(name="A", initializer=tf.constant(2))
b = tf.get_variable(name="B", initializer=tf.constant(3))
c = tf.add(a, b, name="Add")
# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer operation
    sess.run(init_op)
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

Let's look at a very simple example of creating weight and bias variables with proper initialization:

### Example 4:

Create the weight and bias matrices for a fully-connected layer with 2 neuron to another layer with 3 neuron. In this scenario, the weight and bias variables must be of size $[2, 3]$ and 3 respectively.

In [ ]:
# create graph
weights = tf.get_variable(name="W", shape=[2,3], initializer=tf.truncated_normal_initializer(stddev=0.01))
biases = tf.get_variable(name="b", shape=[3], initializer=tf.zeros_initializer())

# add an Op to initialize global variables
init_op = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # run the variable initializer
    sess.run(init_op)
    # now we can run our operations
    W, b = sess.run([weights, biases])
    print('weights = {}'.format(W))
    print('biases = {}'.format(b))

## Placeholder:

Placeholders are more basic than a variable. It is simply a variable that we asign data in a future time. Placeholders are nodes whose value is fed in at execution time. If we have inputs to our network that depend on some external data and we don't want our graph to depend on any real value while developing the graph, placeholders are the datatype we need. In fact, we can build the graph without any data. Therefore, placeholders don't need any initial value; only a datatype (such as float32) and a tensor shape so the graph still knows what to compute with even though it doesn't have any stored values yet.

Some examples of creating placeholders are as follows:

In [ ]:
a = tf.placeholder(tf.float32, shape=[5])
b = tf.placeholder(dtype=tf.float32, shape=None, name=None)
X = tf.placeholder(tf.float32, shape=[None, 784], name='input')
Y = tf.placeholder(tf.float32, shape=[None, 10], name='label') 

Let's run a simple example.

### Example 5:

Create a constant vector and a placeholder and add them together.

In [ ]:
a = tf.constant([5, 5, 5], tf.float32, name='A')
b = tf.placeholder(tf.float32, shape=[3], name='B')
c = tf.add(a, b, name="Add")

with tf.Session() as sess:
      print(sess.run(c))

__InvalidArgumentError: You must feed a value for placeholder tensor 'B' with dtype float and shape  $[3]$ error__


Executing the above code will run into an error. You might have guessed it, its simply because the placeholder is empty and there is no way to add an empty tensor to a constant tensor in the add operation. To solve this, we need to feed an input value to the tensor "b". It can be done by creating a dictionary ("d" in the following code) whose key(s) are the placeholders and their values are the desired value to be passed to the placeholder(s), and feeding it to an argument called "feed_dict". In our example, say we want to pass $[1, 2, 3]$ to the placeholder; the code needs to be modified as:

In [ ]:
a = tf.constant([5, 5, 5], tf.float32, name='A')
b = tf.placeholder(tf.float32, shape=[3], name='B')
c = tf.add(a, b, name="Add")

with tf.Session() as sess:
    # create a dictionary:
    d = {b: [1, 2, 3]}
    # feed it to the placeholder
    print(sess.run(c, feed_dict=d)) 

# Exercises

Create computational graphs and execute the following operations:

1\. $$ \log(x) $$

2\. $$ x^2 +  y$$

3\. $$\frac{\pi}{e^{x}} $$

4\. $$x > y $$
